In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
!pwd

/home/sid/ds_nudge_up/playground


In [93]:
import matplotlib.pyplot as plt

In [94]:
import sys
sys.path += ["../"]

In [95]:
import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

In [96]:
from src.featurize import *

In [97]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_image_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [98]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

In [99]:
%%time
points_gdf_with_stac = fetch_image_refs(points_gdf.loc[:20, :], featurization_params['dask']['n_partitions'], 
                                        featurization_params['satellite_image_params'])

/home/sid/miniconda3/envs/nudge_up/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


CPU times: user 866 ms, sys: 40.5 ms, total: 907 ms
Wall time: 2.25 s


In [100]:
mini_stac_gdf = points_gdf_with_stac[:10]

In [101]:
data_loader = create_data_loader(mini_stac_gdf, satellite_config, featurization_params['batch_size'])

In [102]:
model = RCF(featurization_params['num_features'], featurization_params['kernel_size'], len(satellite_config['bands']))

In [103]:
featurization_params

{'satellite_image_params': {'satellite_name': 'sentinel-2-l2a',
  'search_start': '2021-04-01',
  'search_end': '2022-03-31',
  'stac_output': 'least_cloudy',
  'stac_api': 'planetary-compute'},
 'num_features': 500,
 'kernel_size': 3,
 'batch_size': 5,
 'device': 'cpu',
 'dask': {'n_partitions': 12}}

In [104]:
X_features = create_features(data_loader, featurization_params['num_features'], len(mini_stac_gdf), 
                             model, featurization_params['device'], satellite_config['min_image_edge'])

  0%|                                                                           | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
 50%|█████████████████████████████████▌                                 | 1/2 [00:53<00:53, 53.41s/it]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████| 2/2 [01:35<00:00, 47.77s/it]


In [105]:
X_features

array([[1.09272471e-04, 5.34631603e-04, 3.45778018e-02, ...,
        8.76617253e-01, 3.42695594e-01, 8.57008457e-01],
       [6.75072515e-05, 1.00113072e-01, 1.07603788e-01, ...,
        8.00987422e-01, 3.28515947e-01, 8.57559383e-01],
       [7.99814643e-06, 7.69513696e-02, 2.79357195e-01, ...,
        7.83863008e-01, 1.80771410e-01, 7.95646191e-01],
       ...,
       [4.18336094e-05, 2.65610088e-02, 2.01380104e-01, ...,
        8.71932685e-01, 1.27040088e-01, 8.07325482e-01],
       [0.00000000e+00, 1.09042097e-02, 1.62996724e-02, ...,
        9.12571073e-01, 5.58667898e-01, 9.44035292e-01],
       [5.50447339e-06, 1.56248599e-01, 3.57094228e-01, ...,
        7.71559715e-01, 1.72310799e-01, 7.87246406e-01]])